#Build a model

In [5]:
%%writefile build_model.py
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np



Overwriting build_model.py


In [6]:
!python build_model.py

#Then run the app

In [14]:
%%writefile solarApp.py
from flask import Flask
from flask import request
app = Flask(__name__)
from flask import render_template

# Form page to submit text
#============================================
# create page with a form on it
@app.route('/')
def submission_page():
    #content = 'hello'
    return render_template('template.html')

@app.route('/about')
def about_page():
    #content = 'hello'
    return render_template('about.html')

@app.route('/word_counter', methods=['POST','GET'] )
def word_counter():
#     if request.method == 'POST':
#         return ''
    # get data from request form, the key is the name you set in your form
    data = request.form['user_input']

    # convert data to list
    data = [data]

    import pickle
    import pandas as pd
    import matplotlib.pyplot as plt
    import numpy as np
    

    for doc, category in zip(data, predicted):
        #return('%r => %s' % (doc, categories[category]))
        return render_template('template2.html', doc=doc, category=categories[category])

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=True)

Overwriting solarApp.py


In [ ]:
!python solarApp.py

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
 * Restarting with stat
127.0.0.1 - - [13/Aug/2015 12:01:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Aug/2015 12:01:09] "GET /dist/js/bootstrap.min.js HTTP/1.1" 404 -
127.0.0.1 - - [13/Aug/2015 12:01:09] "GET /assets/js/ie10-viewport-bug-workaround.js HTTP/1.1" 404 -
127.0.0.1 - - [13/Aug/2015 12:03:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Aug/2015 12:03:51] "GET /dist/js/bootstrap.min.js HTTP/1.1" 404 -
127.0.0.1 - - [13/Aug/2015 12:03:51] "GET /assets/js/ie10-viewport-bug-workaround.js HTTP/1.1" 404 -
127.0.0.1 - - [13/Aug/2015 12:06:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Aug/2015 12:06:49] "GET /static/bootstrap-3.3.5-dist/css/bootstrap.min.css HTTP/1.1" 200 -
127.0.0.1 - - [13/Aug/2015 12:06:49] "GET /dist/js/bootstrap.min.js HTTP/1.1" 404 -
127.0.0.1 - - [13/Aug/2015 12:06:49] "GET /assets/js/ie10-viewport-bug-workaround.js HTTP/1.1" 404 -
